# Импорт функции для загрузки данных из файла  

Функция принимает:  

+ путь до файла (строка)  
+ имя таблицы (строка)  
+ начальная/конечная строка с данными (кортеж)  
+ столбцы для выгрузки (кортеж)  
+ столбцы которы преобразуются в числовые данные (кортеж)  


In [ ]:
import string
import pandas as pd
from openpyxl import load_workbook
import os
def load_sheet(path_to_file: str, sheet_name: str, rows_range: tuple, columns_range: tuple, numeric_columns: tuple):
    ''' Load sheet from excel file
    ----------
    Parameters:
    path_to_file : path to excel file (str)
    sheet_name : name of sheet from excel file (str)
    rows_range : tuple with start/end numeric data row indexes (int)
    columns_range : tuple with columns headers, max 26 columns (str)
    numeric_columns : tuple with columns headers, that should be numeric (str)
    -------
    Returns:
    pd_sheet : pandas DataFrame
    '''
    ### Проверяем что файл существует
    if not os.path.exists(path_to_file):
        error_message = f"Не нахожу такого файла: {path_to_file}"
        raise ValueError(error_message)
    ### Формируем заголовок для конечной таблицы
    header = os.path.basename(path_to_file)
    header = header.rsplit(".", maxsplit=1)[0]
    ### Проверяем, что нет повторяющихся столбцов
    if len(set(columns_range)) != len(columns_range):
        error_message = f"Есть повторяющиеся столбцы в {header}"
        raise ValueError(error_message)
    ### Проверяем что столбцы для чисел перечислены в кортеже
    if type(numeric_columns) != tuple:
        error_message = f"Столбцы для чисел должны быть перечислены в кортежах, например так (\"A\",)"
        raise ValueError(error_message)
    ### Загружаем файл
    wb = load_workbook(filename=path_to_file)
    ws_data = wb[sheet_name]
    pd_sheet = pd.DataFrame(ws_data.values)
    wb.close()
    # num_xlsx = pd.ExcelFile(path_to_file)
    # pd_sheet = pd.read_excel(num_xlsx, sheet_name)
    ### Create list with capital letters A,B,C ... J
    col = 26 if len(pd_sheet.columns) > 26 else len(pd_sheet.columns)
    columns = list(string.ascii_uppercase[:col])
    ### Удаляем лишние столбцы
    pd_sheet = pd_sheet.iloc[:, :col]
    pd_sheet.columns = columns
    ### Делаем реиндекс как в Excel
    pd_sheet = pd_sheet.set_index((i+1 for i in pd_sheet.index))
    ### Оставляем только нужные столбцы
    pd_sheet = pd_sheet.loc[:, columns_range]
    ### Удаляем лишние строки
    pd_sheet = pd_sheet.loc[rows_range[0]:rows_range[1],]
    ### Преобразуем данные в числовой формат
    for i in pd_sheet.columns:
        if i in numeric_columns:
            pd_sheet[i] = pd.to_numeric(pd_sheet[i])
    ### Устанавливаем мультииндекс
    pd_sheet.columns = pd.MultiIndex.from_product([[header], pd_sheet.columns])
    print(pd_sheet.head(2))
    print("....................")
    print(pd_sheet.tail(2).to_string(header=False))
    return pd_sheet

# Загружаем настройки

Важен порядок, первым должен идти файл с номенклатурой, потом файлы отчетов.  
Порядок и количество должен сохранятся во всех переменных и быть одинаковым.


In [ ]:
### Настройки
pathes_to_files = ("LFLАпрель/LFL_list.xlsx",
                   "LFLАпрель/ИМ/апрель_2021.xlsx",
                   "LFLАпрель/ИМ/апрель_2021.xlsx",
                   "LFLАпрель/ИМ/апрель_2021.xlsx",
                   "LFLАпрель/ИМ/апрель_2021.xlsx",
                   "LFLАпрель/ИМ/апрель_2021.xlsx")
sheet_names = ("TDSheet", "TDSheet", "TDSheet", "TDSheet", "TDSheet", "TDSheet")
rows_range = ((6, 6834),
              (10, 3441),
              (10, 3441),
              (10, 3441),
              (10, 3441),
              (10, 3441))
columns_range = (("A", "B"),
                 ("A", "H"),
                 ("A", "I"),
                 ("A", "H"),
                 ("A", "I"),
                 ("A", "E", "H", "I"))

# Загружаем номенклатуру


In [ ]:
lfl = load_sheet(pathes_to_files[0], sheet_names[0], rows_range[0], columns_range[0], ("A",))

# Загружаем наши файлы с отчетами


In [ ]:
reports = []
for i in range(1, len(pathes_to_files)):
    reports.append(load_sheet(pathes_to_files[i], sheet_names[i], rows_range[i], columns_range[i], columns_range[i]))

# Присоединить по номенклатуре (ВПР)


In [ ]:
for report in reports:
    result = pd.merge(lfl, report, how="left", left_on=[lfl.columns[0]], right_on=[report.columns[0]])
print(result)